In [1]:
#!/usr/bin/env python3
# fast_clean.py
import os, sys, shutil, stat, concurrent.futures, time

ROOT = r"ALL"          # 要清空的顶层目录
WORKERS = min(32, os.cpu_count() + 4)  # IO 密集，线程可略多于核

def remove_readonly(func, path, exc):
    """Windows 专属：遇到只读文件先改权限再删"""
    os.chmod(path, stat.S_IWRITE)
    func(path)

def wipe_dir(top: str):
    """暴力删整个目录，不经过回收站"""
    shutil.rmtree(top, onerror=remove_readonly)




In [2]:
def main():
    t0 = time.perf_counter()
    # 1. 快速拿到一级子目录列表（不递归）
    dirs = [os.path.join(ROOT, name)
            for name in os.listdir(ROOT)
            if os.path.isdir(os.path.join(ROOT, name))]
    if not dirs:
        print("没有子目录需要删除。")
        return

    # 2. 多线程并行删
    with concurrent.futures.ThreadPoolExecutor(max_workers=WORKERS) as exe:
        futures = [exe.submit(wipe_dir, d) for d in dirs]
        for f in concurrent.futures.as_completed(futures):
            if exc := f.exception():
                print("删失败:", exc)

    print(f"耗时 {time.perf_counter() - t0:.2f} s")

In [3]:
main()

耗时 228.28 s
